In [108]:
import pandas as pd
import re
import numpy as np

In [109]:
filePath = '/Users/lexizhou/Desktop/RR_bonsai_events_TS.csv'

In [110]:
events = pd.read_csv(filePath)
events[:50]
events = events.rename(columns={'9':"event_code", 'Mouse in hall 1': 'event', '246.7670912': 'timestamp'})
events = events.drop(columns=['0'])
events['event'] = events['event'].str.replace('[^\w\s]','')

event_code_dic = events.groupby(['event', 'event_code']).size()
event_code_dic = event_code_dic.to_frame().reset_index().set_index('event_code')
event_code_dict = event_code_dic.to_dict()['event']

events_list = events.values.tolist()

In [111]:
event_code_dict[29]='R2 0 offer'
event_code_dict[27]='R2 noreward tone'

event_code_dict

{11: 'Enter R1',
 23: 'Enter R2',
 35: 'Enter R3',
 47: 'Enter R4',
 9: 'Mouse in hall 1',
 21: 'Mouse in hall 2',
 33: 'Mouse in hall 3',
 45: 'Mouse in hall 4',
 10: 'Mouse in offer zone 1',
 22: 'Mouse in offer zone 2',
 34: 'Mouse in offer zone 3',
 46: 'Mouse in offer zone 4',
 13: 'Quit R1',
 25: 'Quit R2',
 37: 'Quit R3',
 49: 'Quit R4',
 29: 'R2 0 offer',
 27: 'R2 noreward tone',
 17: 'R1 0 offer',
 20: 'R1 100 offer',
 18: 'R1 20 offer',
 19: 'R1 80 offer',
 15: 'R1 noreward tone',
 16: 'R1 reward taken',
 32: 'R2 100 offer',
 30: 'R2 20 offer',
 31: 'R2 80 offer',
 28: 'R2 reward taken',
 41: 'R3 0 offer',
 44: 'R3 100 offer',
 42: 'R3 20 offer',
 43: 'R3 80 offer',
 39: 'R3 noreward tone',
 40: 'R3 reward taken',
 53: 'R4 0 offer',
 56: 'R4 100 offer',
 54: 'R4 20 offer',
 55: 'R4 80 offer',
 51: 'R4 noreward tone',
 52: 'R4 reward taken',
 1: 'Servo 1 open',
 3: 'Servo 2 open',
 5: 'Servo 3 open',
 7: 'Servo 4 open'}

In [117]:
def restaurant_extractor(events_list):
    for i in events_list:
        if len(i) <= 3:
            integers = re.findall('[0-9]+', i[0])
            for j in integers:
                if int(j) > 0 and int(j) < 5:
                    i.append(int(j))

restaurant_extractor(events_list)
events_list[:50]

[['Mouse in offer zone 1', 246.80968959999998, 10, 1, 'offer zone'],
 ['R1 100 offer', 247.0, 20, 1, '100_offer'],
 ['Enter R1', 247.0, 11, 1, 'accept'],
 ['Quit R1', 252.716992, 13, 1, 'quit'],
 ['Mouse in hall 2', 253.3837312, 21, 2, 'hall'],
 ['Mouse in offer zone 2', 253.5054336, 22, 2, 'offer zone'],
 ['R 2 0 offer', 253.51754240000002, 29, 2, '0_offer'],
 ['Enter R2', 255.90618880000002, 23, 2, 'accept'],
 ['R 2 noreward tone', 262.972096, 27, 2, 'noreward'],
 ['Quit R2', 363.43120639999995, 25, 2, 'quit'],
 ['Mouse in hall 3', 363.49783039999994, 33, 3, 'hall'],
 ['Mouse in offer zone 3', 365.9704832, 34, 3, 'offer zone'],
 ['R3 0 offer', 365.98688, 41, 3, '0_offer'],
 ['Enter R3', 367.67199999999997, 35, 3, 'accept'],
 ['R3 noreward tone', 374.7444992, 39, 3, 'noreward'],
 ['Quit R3', 414.3253376, 37, 3, 'quit'],
 ['Mouse in hall 4', 414.49186560000004, 45, 4, 'hall'],
 ['Mouse in offer zone 4', 414.62776319999995, 46, 4, 'offer zone'],
 ['R4 0 offer', 414.6416512, 53, 4, '0_of

In [116]:
def detect_keyword_in_event(events_list):
    """
    event -- List: list of bonsai events
    """
    def detect_keyword(string):
        keyword = None
        if string.__contains__('hall'):
            keyword = 'hall'
        elif string.__contains__('zone'):
            keyword = 'offer zone'
        elif string.__contains__('offer'):
            probability = re.findall('[0-9]+', string)[1]
            keyword = probability + '_offer'
        elif string.__contains__('enter'):
            keyword = 'enter'
        elif string.__contains__('Quit'):
            keyword = 'quit'
        elif string.__contains__('noreward'):
            keyword = 'noreward'
        elif string.__contains__('taken'):
            keyword = 'taken'
        elif string.__contains__('servo'):
            keyword = 'servo'
        elif string.__contains__('Enter'):
            keyword = 'enter'
        return keyword
    new_events_list = []
    for i in events_list:
        if len(i) < 5:
            i_description = i[0]
            i += [detect_keyword(i_description)]
            new_events_list.append(i)
        else:
            new_events_list.append(i)
    return new_events_list
        
events_list_with_keyword = detect_keyword_in_event(events_list)
events_list_with_keyword[:50]

[['Mouse in offer zone 1', 246.80968959999998, 10, 1, 'offer zone'],
 ['R1 100 offer', 247.0, 20, 1, '100_offer'],
 ['Enter R1', 247.0, 11, 1, 'accept'],
 ['Quit R1', 252.716992, 13, 1, 'quit'],
 ['Mouse in hall 2', 253.3837312, 21, 2, 'hall'],
 ['Mouse in offer zone 2', 253.5054336, 22, 2, 'offer zone'],
 ['R 2 0 offer', 253.51754240000002, 29, 2, '0_offer'],
 ['Enter R2', 255.90618880000002, 23, 2, 'accept'],
 ['R 2 noreward tone', 262.972096, 27, 2, 'noreward'],
 ['Quit R2', 363.43120639999995, 25, 2, 'quit'],
 ['Mouse in hall 3', 363.49783039999994, 33, 3, 'hall'],
 ['Mouse in offer zone 3', 365.9704832, 34, 3, 'offer zone'],
 ['R3 0 offer', 365.98688, 41, 3, '0_offer'],
 ['Enter R3', 367.67199999999997, 35, 3, 'accept'],
 ['R3 noreward tone', 374.7444992, 39, 3, 'noreward'],
 ['Quit R3', 414.3253376, 37, 3, 'quit'],
 ['Mouse in hall 4', 414.49186560000004, 45, 4, 'hall'],
 ['Mouse in offer zone 4', 414.62776319999995, 46, 4, 'offer zone'],
 ['R4 0 offer', 414.6416512, 53, 4, '0_of

In [140]:
class BonsaiEvent:
    
    def __init__(self, event):
        self.item = self
        self.event_description = event[0]
        self.timestamp = event[1]
        self.event_code = event[2]
        self.restaurant = event[3]
        self.keyword = event[4]
        self.prev = None
        self.next = None
        
    
class Trial:
    """
    Series of bonsai events in the same restaurant structured as 
    a doubly linked list
    """
    valid = False
    def __init__(self, events):
        """
        events -- DLL: list of bonsai_event objects
        """
        self.tone_prob
        self.
        
def restaurant_chuncker(events_list):
    current_restaurant = events_list[0][-1]
    visit = [events_list[0]]
    new_event_list = []
    for i in range(len(events_list[1:])):
        if events_list[i][-1] == current_restaurant:
            visit.append(events_list[i])
        else:
            current_restaurant = events_list[i][-1]
            new_event_list.append(visit)
            visit = [events_list[i+1]]
    return new_event_list

events_by_restaurants = restaurant_chuncker(events_list)
events_by_restaurants[0]
        
    
    def __init__()
    
class Lap:
    """A doubly linked list of trials"""
    
class Session:
    """A doubly linked list of laps"""

In [141]:
class DoublyLinkedList:
    """A doubly linked list data structure"""
    def __init__(self):
        self.head = None
        
    def write_dataFrame(self):
        assert self.head is not None
        n = self.head
        list_of_objects = []
        while n is not None:
            list_of_objects.append(n.item)
            n = n.next
        dataFrame = pd.DataFrame([obj.__dict__ for obj in list_of_objects])
        dataFrame = dataFrame.drop(columns=['item', 'prev', 'next'])
        return dataFrame
    
    def add_to_end(self, item):
        if self.head is None:
            new_item = item
            self.head = new_item
            return
        n = self.head
        while n.next is not None:
            n = n.next
        new_item = item
        n.next = new_item
        new_item.prev = n
    
    def add_to_start(self, item):
        if self.head is None:
            new_item = item
            self.head = new_item
            return
        new_item = item
        new_item.next = self.head
        self.head.prev = new_item
        self.head = new_item

In [143]:
list_of_event_objects = DoublyLinkedList()
for i in events_list:
    event_object = BonsaiEvent(i)
    list_of_event_objects.add_to_end(event_object)

In [14]:
def identify_direction_blocks(events):
    prev = events[0][0][-1]
    block = [events[0]]
    new_event_list = []
    for i in range(len(events[1:])):
        i_r = events[i][0][3]
        if i_r >= prev:
            prev = i_r
            block.append(events[i])
        elif prev == 4 and i_r == 1:
            prev = i_r
            block.append(events[i])
        else:
            prev = i_r
            new_event_list.append(block)
            block = [events[i]]
    return new_event_list

events_by_blocks = identify_direction_blocks(events_by_restaurants)

In [ ]:
for i in events_by_blocks:
    